In [1]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [2]:
list=[['Bob', 5.62,'juice'],
      ['Sue',  0.85, 'milk'],
      ['Joe',  1.04, 'eggs']]

df=spark.createDataFrame(list,['name','amount','item'])

df.show()


+----+------+-----+
name|amount| item|
+----+------+-----+
 Bob| 5.62|juice|
 Sue| 0.85| milk|
 Joe| 1.04| eggs|
+----+------+-----+

In [3]:
df.filter(~df.item.isin(['milk','eggs'])).show()

+----+------+-----+
name|amount| item|
+----+------+-----+
 Bob| 5.62|juice|
+----+------+-----+

In [4]:
df = df.filter(~df.item.isin(['milk','eggs']))

File "<command-2321691597675911>" , line 1 
 df = df.filter(~df.item.isin(['milk','eggs']) 
 ^
 SyntaxError : unexpected EOF while parsing

In [5]:
list=[['A' ,      '2019-12-15',      '2020-04-30'],
     ['B' ,      '2020-03-03'  ,    '2020-04-30']]

df=spark.createDataFrame(list,['ID','Start_Date','End_Date'])

df.show()

+---+----------+----------+
 ID|Start_Date| End_Date|
+---+----------+----------+
 A|2019-12-15|2020-04-30|
 B|2020-03-03|2020-04-30|
+---+----------+----------+

In [6]:
from pyspark.sql import functions as F

df.withColumn("Id_period", F.explode(F.expr("""transform(sequence(to_date(start_date),to_date(end_date)\
                                                         ,interval 1 month),x-> date_format(x,'yyyyMM'))"""))).show()

+---+----------+----------+---------+
 ID|Start_Date| End_Date|Id_period|
+---+----------+----------+---------+
 A|2019-12-15|2020-04-30| 201912|
 A|2019-12-15|2020-04-30| 202001|
 A|2019-12-15|2020-04-30| 202002|
 A|2019-12-15|2020-04-30| 202003|
 A|2019-12-15|2020-04-30| 202004|
 B|2020-03-03|2020-04-30| 202003|
 B|2020-03-03|2020-04-30| 202004|
+---+----------+----------+---------+

In [7]:
from pyspark.sql import functions as F
df.withColumn("monthsDiff", F.months_between("End_Date", "Start_Date"))\
    .withColumn("repeat", F.expr("split(repeat(',', monthsDiff), ',')"))\
     .select("*", F.posexplode("repeat").alias("date", "val"))\
    .withColumn("Id_period", F.expr("""date_format(add_months(Start_Date, date),'yyyyMM')"""))\
    .drop("repeat","val","monthsDiff","date").show()

+---+----------+----------+---------+
 ID|Start_Date| End_Date|Id_period|
+---+----------+----------+---------+
 A|2019-12-15|2020-04-30| 201912|
 A|2019-12-15|2020-04-30| 202001|
 A|2019-12-15|2020-04-30| 202002|
 A|2019-12-15|2020-04-30| 202003|
 A|2019-12-15|2020-04-30| 202004|
 B|2020-03-03|2020-04-30| 202003|
 B|2020-03-03|2020-04-30| 202004|
+---+----------+----------+---------+

In [8]:
ID      Start_Date      End_Date       ID_period
A       2019-12-15      2020-04-30     201912
A       2019-12-15      2020-04-30     202001
A       2019-12-15      2020-04-30     202002
A       2019-12-15      2020-04-30     202003
A       2019-12-15      2020-04-30     202004
B       2020-03-03      2020-04-30     202003
B       2020-03-03      2020-04-30     202004